In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import Prepare_Data
import pyarrow 
import numpy as np

#autoreload download #pip install ipython
%load_ext autoreload
%autoreload 2

file_path_id_test="./data_test/top_5_percent_ids.csv"
# Sti til usa_dsf.parquet
file_path_usa_dsf_test = "./data_test/usa_dsf_test.parquet"
file_path_usa_test = "./data_test/usa_test.parquet"
file_path_cluster_labels = "./Data/Cluster Labels.csv"
file_path_factor_details = "./Data/Factor Details.xlsx"


file_path_usa = "./Data/usa.parquet"
file_path_rvol_252 = "./Data/rvol_252.csv"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
def merge_rvol_data(file_path_usa, file_path_rvol_252):
    """
    Indlæser USA data fra Parquet og rvol_252 fra CSV, konverterer relevante kolonner,
    merger dataene på 'id' og 'eom', udskriver antal ikke-NaN værdier før og efter merge,
    og gemmer den merged dataframe som 'usa_rvol.parquet' i ./Data/ mappen.
    
    Args:
    - file_path_usa (str): Sti til Parquet-filen.
    - file_path_rvol_252 (str): Sti til CSV-filen.
    Examp
    Returns:
    - df (pd.DataFrame): Den merged dataframe.
    """
    
    # Indlæs data
    df = pd.read_parquet(file_path_usa, engine='pyarrow')
    print("Fil indlæst med succes. Antal rækker før filtrering:", len(df))

    rvol_252 = pd.read_csv(file_path_rvol_252)

    # Konverter 'id' til samme datatype (int64)
    df['id'] = df['id'].astype('int64')

    # Konverter 'eom' til datetime
    df['eom'] = pd.to_datetime(df['eom'])
    rvol_252['eom'] = pd.to_datetime(rvol_252['eom'])

    # Udskriv antal ikke-NaN værdier i rvol_252d i rvol_252 før merge
    num_non_nan_rvol_252 = rvol_252['rvol_252d'].notna().sum()
    print(f"Antal ikke-NaN værdier i 'rvol_252d' i rvol_252 før merge: {num_non_nan_rvol_252}")

    # Merge dataene baseret på 'id' og 'eom'
    df = df.merge(rvol_252[['id', 'eom', 'rvol_252d']], on=['id', 'eom'], how='left')

    # Udskriv antal ikke-NaN værdier i 'rvol_252d' efter merge
    num_non_nan_df = df['rvol_252d'].notna().sum()
    print(f"Antal ikke-NaN værdier i 'rvol_252d' i df efter merge: {num_non_nan_df}")

    # Tjek om vi nu får nogle ikke-NaN værdier
    print(df[['id', 'eom', 'rvol_252d']].head(10))

    # Gem den merged dataframe som Parquet i ./Data/ mappen
    save_path = "./Data/usa_rvol.parquet"
    df.to_parquet(save_path, engine="pyarrow", index=False)

    print("Filen 'usa_rvol.parquet' er gemt succesfuldt i ./Data mappen!")

    return df  # Returnér den merged dataframe

In [21]:
file_path_usa = "./Data/usa.parquet"
file_path_rvol_252 = "./Data/rvol_252.csv"

# Indlæs data
df = pd.read_parquet(file_path_usa, engine='pyarrow')
print("Fil indlæst med succes. Antal rækker før filtrering:", len(df))

rvol_252 = pd.read_csv(file_path_rvol_252)

# Konverter 'id' til samme datatype (int64)
df['id'] = df['id'].astype('int64')

# Konverter 'eom' til datetime
df['eom'] = pd.to_datetime(df['eom'])
rvol_252['eom'] = pd.to_datetime(rvol_252['eom'])

# Udskriv antal ikke-NaN værdier i rvol_252d i rvol_252 før merge
num_non_nan_rvol_252 = rvol_252['rvol_252d'].notna().sum()
print(f"Antal ikke-NaN værdier i 'rvol_252d' i rvol_252 før merge: {num_non_nan_rvol_252}")

# Merge dataene baseret på 'id' og 'eom'
df = df.merge(rvol_252[['id', 'eom', 'rvol_252d']], on=['id', 'eom'], how='left')

# Udskriv antal ikke-NaN værdier i 'rvol_252d' efter merge
num_non_nan_df = df['rvol_252d'].notna().sum()
print(f"Antal ikke-NaN værdier i 'rvol_252d' i df efter merge: {num_non_nan_df}")

# Tjek om vi nu får nogle ikke-NaN værdier
print(df[['id', 'eom', 'rvol_252d']].head(10))

Fil indlæst med succes. Antal rækker før filtrering: 1373911
Antal ikke-NaN værdier i 'rvol_252d' i rvol_252 før merge: 2199588
Antal ikke-NaN værdier i 'rvol_252d' i df efter merge: 1353529
      id        eom  rvol_252d
0  10006 1952-08-31   0.012671
1  10006 1952-09-30   0.012712
2  10006 1952-10-31   0.011661
3  10006 1952-11-30   0.011523
4  10006 1952-12-31   0.011812
5  10006 1953-01-31   0.011484
6  10006 1953-02-28   0.011835
7  10006 1953-03-31   0.012331
8  10006 1953-04-30   0.012696
9  10006 1953-05-31   0.013877


In [8]:
df = pd.read_parquet(file_path_usa, engine='pyarrow')
print("Fil indlæst med succes. Antal rækker før filtrering:", len(df))
df.head(10)

Fil indlæst med succes. Antal rækker før filtrering: 1373911


,eom_ret,id,eom,valid_ret,excntry,me,sic,size_grp,ret_exc_lead1m,niq_su,...,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety,ctff_test
0,1952-09-30,10006,1952-08-31,True,USA,23.885125,3740,small,-0.056756,NaN,...,0.100695,0.084642,0.038812,0.637818,0.669017,NaN,0.274990,NaN,-0.329044,False
1,1952-10-31,10006,1952-09-30,True,USA,22.076500,3740,small,-0.046028,NaN,...,0.100695,0.084642,0.038812,0.653567,0.627405,NaN,0.297905,NaN,-0.364834,False
2,1952-11-30,10006,1952-10-31,True,USA,21.088000,3740,small,0.112232,NaN,...,0.100695,0.084642,0.038812,0.497829,0.625762,NaN,0.342228,NaN,-0.244439,False
3,1952-12-31,10006,1952-11-30,True,USA,23.476875,3740,small,0.050879,NaN,...,0.100695,0.084642,0.038812,0.462569,0.626470,NaN,0.357438,NaN,-0.196596,False
4,1953-01-31,10006,1952-12-31,True,USA,24.218250,3740,small,-0.035634,NaN,...,0.100695,0.084642,0.038812,0.575019,0.667461,NaN,0.395463,NaN,-0.223198,False
5,1953-02-28,10006,1953-01-31,True,USA,23.394500,3740,small,0.111274,NaN,...,0.100695,0.084642,0.038812,0.654311,0.663235,NaN,0.390803,NaN,-0.153599,False
6,1953-03-31,10006,1953-02-28,True,USA,26.030500,3740,small,-0.058854,NaN,...,0.100695,0.084642,0.038812,0.565347,0.662090,NaN,0.428745,NaN,-0.166422,False
7,1953-04-30,10006,1953-03-31,True,USA,24.053500,3740,small,0.042885,NaN,...,0.100695,0.084642,0.038812,0.676796,0.703290,NaN,0.439165,NaN,-0.223840,False
8,1953-05-31,10006,1953-04-30,True,USA,25.124375,3740,small,0.263852,NaN,...,0.100695,0.084642,0.038812,0.673364,0.664946,NaN,0.557875,NaN,-0.081148,False
9,1953-06-30,10006,1953-05-31,True,USA,31.796750,3740,large,-0.102834,NaN,...,0.100695,0.084642,0.038812,0.630026,0.667311,NaN,0.567948,NaN,-0.193354,False


In [11]:
print(df[['id', 'eom']].dtypes)
print(rvol_252[['id', 'eom']].dtypes)


id      int32
eom    object
dtype: object
id      int64
eom    object
dtype: object


In [18]:
rvol_252 = pd.read_csv(file_path_rvol_252)
rvol_252.head(10)
print(rvol_252[['id', 'eom', 'rvol_252d']].head(10))

      id         eom  rvol_252d
0  10006  1952-03-31   0.014800
1  10006  1952-04-30   0.014024
2  10006  1952-05-31   0.013877
3  10006  1952-06-30   0.013516
4  10006  1952-07-31   0.013197
5  10006  1952-08-31   0.012671
6  10006  1952-09-30   0.012712
7  10006  1952-10-31   0.011661
8  10006  1952-11-30   0.011523
9  10006  1952-12-31   0.011812


In [14]:
print(rvol_252.columns)


Index(['obs_main', 'exch_main', 'common', 'primary_sec', 'permno', 'date',
       'id', 'excntry', 'eom', 'rvol_252d'],
      dtype='object')


In [15]:
df = df.merge(rvol_252[['id', 'eom', 'rvol_252d']], on=['id', 'eom'], how='left')


In [17]:
print(df[['id', 'eom', 'rvol_252d']].head(10))

      id        eom  rvol_252d
0  10006 1952-08-31   0.012671
1  10006 1952-09-30   0.012712
2  10006 1952-10-31   0.011661
3  10006 1952-11-30   0.011523
4  10006 1952-12-31   0.011812
5  10006 1953-01-31   0.011484
6  10006 1953-02-28   0.011835
7  10006 1953-03-31   0.012331
8  10006 1953-04-30   0.012696
9  10006 1953-05-31   0.013877


In [19]:
df.to_parquet("./Data/usa_rvol.parquet", engine="pyarrow", index=False)

print("Filen 'usa_rvol.parquet' er gemt succesfuldt i ./Data mappen!")

Filen 'usa_rvol.parquet' er gemt succesfuldt i ./Data mappen!


In [13]:
# Konverter 'id' til samme datatype (int64)
df['id'] = df['id'].astype('int64')

# Konverter 'eom' til datetime
df['eom'] = pd.to_datetime(df['eom'])
rvol_252['eom'] = pd.to_datetime(rvol_252['eom'])

# Merge dataene baseret på 'id' og 'eom'
df = df.merge(rvol_252[['id', 'eom', 'rvol_252d']], on=['id', 'eom'], how='left')

# Tjek om vi nu får nogle ikke-NaN værdier
print(df[['id', 'eom', 'rvol_252d']].head(10))


KeyError: "['rvol_252d'] not in index"

In [12]:
print(df[df['id'] == 10006][['id', 'eom']].head(5))
print(rvol_252[rvol_252['id'] == 10006][['id', 'eom']].head(5))


      id         eom
0  10006  1952-08-31
1  10006  1952-09-30
2  10006  1952-10-31
3  10006  1952-11-30
4  10006  1952-12-31
      id         eom
0  10006  1952-03-31
1  10006  1952-04-30
2  10006  1952-05-31
3  10006  1952-06-30
4  10006  1952-07-31


In [24]:
features = [
  "age",                 "aliq_at",             "aliq_mat",            "ami_126d",
  "at_be",               "at_gr1",              "at_me",               "at_turnover",
  "be_gr1a",             "be_me",               "beta_60m",            "beta_dimson_21d",
  "betabab_1260d",       "betadown_252d",       "bev_mev",             "bidaskhl_21d",
  "capex_abn",           "capx_gr1",            "capx_gr2",            "capx_gr3",
  "cash_at",             "chcsho_12m",          "coa_gr1a",            "col_gr1a",
  "cop_at",              "cop_atl1",            "corr_1260d",          "coskew_21d",
  "cowc_gr1a",           "dbnetis_at",          "debt_gr3",            "debt_me",
  "dgp_dsale",           "div12m_me",           "dolvol_126d",         "dolvol_var_126d",
  "dsale_dinv",          "dsale_drec",          "dsale_dsga",          "earnings_variability",
  "ebit_bev",            "ebit_sale",           "ebitda_mev",          "emp_gr1",
  "eq_dur",              "eqnetis_at",          "eqnpo_12m",           "eqnpo_me",
  "eqpo_me",             "f_score",             "fcf_me",              "fnl_gr1a",
  "gp_at",               "gp_atl1",             "ival_me",             "inv_gr1",
  "inv_gr1a",            "iskew_capm_21d",      "iskew_ff3_21d",       "iskew_hxz4_21d",
  "ivol_capm_21d",       "ivol_capm_252d",      "ivol_ff3_21d",        "ivol_hxz4_21d",
  "kz_index",            "lnoa_gr1a",           "lti_gr1a",            "market_equity",
  "mispricing_mgmt",     "mispricing_perf",     "ncoa_gr1a",           "ncol_gr1a",
  "netdebt_me",          "netis_at",            "nfna_gr1a",           "ni_ar1",
  "ni_be",               "ni_inc8q",            "ni_ivol",             "ni_me",
  "niq_at",              "niq_at_chg1",         "niq_be",              "niq_be_chg1",
  "niq_su",              "nncoa_gr1a",          "noa_at",              "noa_gr1a",
  "o_score",             "oaccruals_at",        "oaccruals_ni",        "ocf_at",
  "ocf_at_chg1",         "ocf_me",              "ocfq_saleq_std",      "op_at",
  "op_atl1",             "ope_be",              "ope_bel1",            "opex_at",
  "pi_nix",              "ppeinv_gr1a",         "prc",                 "prc_highprc_252d",
  "qmj",                 "qmj_growth",          "qmj_prof",            "qmj_safety",
  "rd_me",               "rd_sale",             "rd5_at",              "resff3_12_1",
  "resff3_6_1",          "ret_1_0",             "ret_12_1",            "ret_12_7",
  "ret_3_1",             "ret_6_1",             "ret_60_12",           "ret_9_1",
  "rmax1_21d",           "rmax5_21d",           "rmax5_rvol_21d",      "rskew_21d",
  "rvol_21d",            "sale_bev",            "sale_emp_gr1",        "sale_gr1",
  "sale_gr3",            "sale_me",             "saleq_gr1",           "saleq_su",
  "seas_1_1an",          "seas_1_1na",          "seas_11_15an",        "seas_11_15na",
  "seas_16_20an",        "seas_16_20na",        "seas_2_5an",          "seas_2_5na",
  "seas_6_10an",         "seas_6_10na",         "sti_gr1a",            "taccruals_at",
  "taccruals_ni",        "tangibility",         "tax_gr1a",            "turnover_126d",
  "turnover_var_126d",   "z_score",             "zero_trades_126d",    "zero_trades_21d",
  "zero_trades_252d",
  "rvol_252d"
]

In [25]:
import pandas as pd

def load_and_prepare_data(file_path, features):
    """
    Indlæser og forbereder rådata fra en Parquet-fil.
    
    Args:
        file_path (str): Stien til Parquet-filen (USA testfil).
        features (list): Liste over features, der skal indlæses.
    
    Returns:
        pd.DataFrame: DataFrame med forberedte data.
    """
    # Liste over kolonner, vi gerne vil have
    desired_cols = ["id", "eom", "sic", "ff49", "size_grp", "me", "crsp_exchcd", "rvol_252d", "dolvol_126d"] + features
    
    # Tjek hvilke kolonner der faktisk findes i Parquet-filen
    actual_cols = pd.read_parquet(file_path, engine="pyarrow").columns
    cols_to_load = [col for col in desired_cols if col in actual_cols]
    
    if not cols_to_load:
        raise ValueError("Ingen af de ønskede kolonner findes i Parquet-filen.")
    
    # Indlæs kun de kolonner, der faktisk findes
    data = pd.read_parquet(file_path, engine="pyarrow", columns=cols_to_load)
    
    # Filtrér observationer
    data = data[data["id"] <= 99999]
    
    # Konverter datoformat for 'eom'
    data["eom"] = pd.to_datetime(data["eom"], errors="coerce")
    
    return data


In [26]:
data = load_and_prepare_data(file_path_usa_test, features)
data.head()

,id,eom,sic,size_grp,me,dolvol_126d,age,aliq_at,aliq_mat,ami_126d,...,taccruals_at,taccruals_ni,tangibility,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d
0,10104,2010-01-31,7372,mega,115558.73052,6.798938e+08,301,0.526422,0.232151,0.000016,...,-0.040247,-0.378191,0.384579,-0.000151,0.006062,0.439375,4.269989,0.002792,0.002780,0.002652
1,10104,2010-02-28,7372,mega,123619.74809,7.094750e+08,302,0.526422,0.249602,0.000016,...,-0.040247,-0.378191,0.384579,-0.000151,0.006243,0.425637,4.269989,0.002704,0.002416,0.002676
2,10104,2010-03-31,7372,mega,129040.82501,6.982804e+08,303,0.570962,0.230516,0.000015,...,-0.009307,-0.086350,0.384579,0.000186,0.005967,0.356592,4.405333,0.002753,0.002751,0.002807
3,10104,2010-04-30,7372,mega,130101.18774,6.802223e+08,304,0.570962,0.218079,0.000015,...,-0.009307,-0.086350,0.384579,0.000186,0.005590,0.344846,4.405333,0.002935,0.003165,0.002890
4,10104,2010-05-31,7372,mega,113436.81847,7.368972e+08,305,0.570962,0.215020,0.000015,...,-0.009307,-0.086350,0.384579,0.000186,0.006023,0.361901,4.405333,0.002833,0.002529,0.002844


In [20]:
def calculate_derived_columns(data, pi):
    """
    Beregner afledte kolonner.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        pi (float): En skalar værdi, der bruges i beregningen af lambda.
    
    Returns:
        pd.DataFrame: DataFrame med nye kolonner.
    """
    # Kontrollér inputtyper
    if not isinstance(pi, (int, float)):
        raise ValueError("'pi' skal være en numerisk værdi (int eller float).")
    
    # Kontrollér for nødvendige kolonner
    required_cols = ["dolvol_126d", "rvol_252d"]
    for col in required_cols:
        if col not in data.columns:
            raise KeyError(f"Kolonnen '{col}' mangler i DataFrame.")

    # Debugging: Tjek for null-værdier
    for col in required_cols:
        if data[col].isnull().any():
            raise ValueError(f"Kolonnen '{col}' indeholder manglende værdier.")
    
    # Debugging: Tjek for 0-værdier i 'dolvol_126d'
    if (data["dolvol_126d"] == 0).any():
        raise ValueError("Kolonnen 'dolvol_126d' indeholder værdier lig 0, hvilket vil føre til divisionsfejl.")

    # Beregn afledte kolonner
    print("Beregner 'dolvol', 'lambda' og 'rvol_m'...")
    data["dolvol"] = data["dolvol_126d"]  # Kopiér dolvol_126d til dolvol
    data["lambda"] = 2 / data["dolvol"] * pi  # Beregn lambda
    data["rvol_m"] = data["rvol_252d"] * np.sqrt(21)  # Beregn rvol_m
    
    print("Beregnede kolonner 'dolvol', 'lambda' og 'rvol_m' succesfuldt.")
    return data

In [19]:
    print("Kolonner i DataFrame:")
    for col in data.columns:
        print(col)

Kolonner i DataFrame:
id
eom
sic
size_grp
me
dolvol_126d
age
aliq_at
aliq_mat
ami_126d
at_be
at_gr1
at_me
at_turnover
be_gr1a
be_me
beta_60m
beta_dimson_21d
betabab_1260d
betadown_252d
bev_mev
bidaskhl_21d
capex_abn
capx_gr1
capx_gr2
capx_gr3
cash_at
chcsho_12m
coa_gr1a
col_gr1a
cop_at
cop_atl1
corr_1260d
coskew_21d
cowc_gr1a
dbnetis_at
debt_gr3
debt_me
dgp_dsale
div12m_me
dolvol_126d
dolvol_var_126d
dsale_dinv
dsale_drec
dsale_dsga
earnings_variability
ebit_bev
ebit_sale
ebitda_mev
emp_gr1
eq_dur
eqnetis_at
eqnpo_12m
eqnpo_me
eqpo_me
f_score
fcf_me
fnl_gr1a
gp_at
gp_atl1
ival_me
inv_gr1
inv_gr1a
iskew_capm_21d
iskew_ff3_21d
iskew_hxz4_21d
ivol_capm_21d
ivol_capm_252d
ivol_ff3_21d
ivol_hxz4_21d
kz_index
lnoa_gr1a
lti_gr1a
market_equity
mispricing_mgmt
mispricing_perf
ncoa_gr1a
ncol_gr1a
netdebt_me
netis_at
nfna_gr1a
ni_ar1
ni_be
ni_inc8q
ni_ivol
ni_me
niq_at
niq_at_chg1
niq_be
niq_be_chg1
niq_su
nncoa_gr1a
noa_at
noa_gr1a
o_score
oaccruals_at
oaccruals_ni
ocf_at
ocf_at_chg1
ocf_me
ocfq

In [21]:
calculate_derived_columns(data, 0.1)

KeyError: "Kolonnen 'rvol_252d' mangler i DataFrame."

In [22]:
# Tjek for lignende kolonnenavne
print("Kolonner i DataFrame:")
for col in data.columns:
    if "rvol" in col.lower():
        print(f"- {col}")


Kolonner i DataFrame:
- rmax5_rvol_21d
- rvol_21d


In [ ]:
def merge_data(data, data_ret_ld1, wealth):
    """
    Slår data sammen med afkast og wealth.
    
    Args:
        data (pd.DataFrame): Hoveddataset.
        data_ret_ld1 (pd.DataFrame): Afkastdata.
        wealth (pd.DataFrame): Wealth-data.
    
    Returns:
        pd.DataFrame: Kombineret DataFrame.
    """
    # Merge afkastdata
    data = pd.merge(data, data_ret_ld1, on=["id", "eom"], how="left")
    
    # Merge wealth data
    wealth["eom"] = wealth["eom"] + pd.offsets.MonthBegin(2) - pd.Timedelta(days=1)
    data = pd.merge(data, wealth[["eom", "mu_ld1"]].rename(columns={"mu_ld1": "mu_ld0"}), on="eom", how="left")
    
    return data


In [ ]:
def screen_data(data, settings):
    """
    Filtrerer data baseret på forskellige kriterier.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        settings (dict): Indstillinger for screens.
    
    Returns:
        pd.DataFrame: Filtreret DataFrame.
    """
    # Start antal observationer og markedsværdi
    n_start = len(data)
    me_start = data["me"].sum()
    
    # NYSE screen
    if settings["screens"]["nyse_stocks"]:
        data = data[data["crsp_exchcd"] == 1]
    
    # Dato screen
    data = data[(data["eom"] >= settings["screens"]["start"]) & (data["eom"] <= settings["screens"]["end"])]
    
    # Fjern observationer med manglende eller ugyldige værdier
    data = data[data["me"].notna() & data["tr_ld0"].notna() & data["tr_ld1"].notna()]
    data = data[data["dolvol"].notna() & (data["dolvol"] > 0)]
    data = data[data["sic"].notna()]
    
    return data


In [ ]:
def standardize_and_impute_features(data, features, settings):
    """
    Standardiserer og imputerer features.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        features (list): Liste over features.
        settings (dict): Indstillinger for standardisering og imputering.
    
    Returns:
        pd.DataFrame: DataFrame med standardiserede og imputerede features.
    """
    # Standardisering
    if settings["feat_prank"]:
        for f in features:
            data[f] = data[f].rank(pct=True)
            data[f] = data[f].fillna(0)  # Sæt manglende værdier til 0
    
    # Imputering
    if settings["feat_impute"]:
        for f in features:
            if settings["feat_prank"]:
                data[f] = data[f].fillna(0.5)  # Median for standardiserede værdier
            else:
                data[f] = data.groupby("eom")[f].transform(lambda x: x.fillna(x.median()))
    
    return data


In [ ]:
def classify_industries(data):
    """
    Klassificerer observationer baseret på SIC-koder.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
    
    Returns:
        pd.DataFrame: DataFrame med klassificerede industrier.
    """
    conditions = [
        data["sic"].between(100, 999) | data["sic"].between(2000, 2399),
        data["sic"].between(2500, 2519) | data["sic"].between(3630, 3659),
        data["sic"].between(2520, 2589) | data["sic"].between(2600, 2699),
        data["sic"].between(1200, 1399) | data["sic"].between(2900, 2999)
    ]
    choices = ["NoDur", "Durbl", "Manuf", "Enrgy"]
    data["ff12"] = np.select(conditions, choices, default="Other")
    
    return data


In [ ]:
def validate_and_finalize(data, pf_set):
    """
    Validerer og afslutter datasættet.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        pf_set (dict): Indstillinger for validering.
    
    Returns:
        pd.DataFrame: Valideret DataFrame.
    """
    lb = pf_set["lb_hor"] + 1
    data["eom_lag"] = data.groupby("id")["eom"].shift(lb)
    data["month_diff"] = ((data["eom"] - data["eom_lag"]).dt.days / 30).round()
    data["valid_data"] = (data["month_diff"] == lb) & data["month_diff"].notna()
    data = data[data["valid_data"]]
    
    return data


In [42]:
def prepare_daily_returns(file_path, data):
    """
    Forbereder daglige afkastdata.
    
    Args:
        file_path (str): Stien til CSV-filen med daglige data. usa_dsf
        chars (pd.DataFrame): Filtreret DataFrame med gyldige observationer.
    
    Returns:
        pd.DataFrame: DataFrame med daglige afkast.
    """
    usecols=["id", "date", "ret_exc"]
    daily = pd.read_parquet(file_path, engine="pyarrow", columns=usecols)
    daily = daily[daily["ret_exc"].notna() & daily["id"].isin(data["id"].unique())]
    daily["date"] = pd.to_datetime(daily["date"], format="%Y%m%d")
    daily["eom"] = daily["date"] + pd.offsets.MonthEnd(0)
    
    return daily


In [43]:
prepare_daily_returns("./data_test/usa_dsf_test.parquet", data)

,id,date,ret_exc,eom
0,10104,2010-02-01,0.006938,2010-02-28
1,10104,2010-02-02,0.023255,2010-02-28
2,10104,2010-02-03,-0.000421,2010-02-28
3,10104,2010-02-04,-0.026948,2010-02-28
4,10104,2010-02-05,0.019039,2010-02-28
...,...,...,...,...
602507,92655,2023-12-22,0.000624,2023-12-31
602508,92655,2023-12-26,-0.000741,2023-12-31
602509,92655,2023-12-27,0.005104,2023-12-31
602510,92655,2023-12-28,0.003833,2023-12-31
